In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [2]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'
ab_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\repo\\base'

In [2]:
#stationär
data_dir = 'c:\\Users\\erikw\\Desktop\\Exjobb kod\\data'
ab_dir = 'c:\\Users\\erikw\\Desktop\\Exjobb kod\\base'

In [ ]:
#saga
data_dir = "/home/aeerik/data/raw/"


CONFIG

In [11]:
# Budget config file
include_pheno = False
threshold_year = 1970
data_path = data_dir
ab_path = ab_dir
max_length = [51,81]
mask_prob = 0.15
embedding_dim = 32
drop_prob = 0.2

#Encoder
dim_emb = 128
dim_hidden = 128
attention_heads = 8 

#BERT
num_encoders = 2

#trainer
epochs = 5
batch_size = 32
lr = 0.001
stop_patience = 5


Vocabulary

In [ ]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
include_pheno = False
vocabulary = vocab_geno(NCBI, include_pheno)
vocab = vocab_pheno(ab_df)
print(len(vocabulary))
print(len(vocab))


In [4]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from data_preprocessing import data_loader
from create_dataset import NCBIDataset

include_pheno = True
threshold_year = 1970

data_path = data_dir
ab_path = ab_dir

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)

max_length = [51,81]
mask_prob = 0.25
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

test_set = NCBIDataset(NCBI, vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
test_set.prepare_dataset()

In [ ]:
sequences = deepcopy(NCBI['AST_phenotypes'].tolist())
max_seq_len = [max_length[0],max_length[1]]
list_idx = []
list_SR = []
for i in range(len(sequences)):
    current_seq = sequences[i]
    current_idxs = []
    current_SRs = []
    for j in range(len(current_seq)):
        item = current_seq[j].split('=')
        abs = item[0]   
        sr = item[1]
        current_idxs.append(vocabulary_pheno.lookup_indices([abs]))
        for k in range(len(sr)):
            if sr == 'R':
                current_SRs.append(1)
            else:
                current_SRs.append(0)

    if len(current_idxs) != len(current_SRs):
        print("current sequence:",current_seq, "\n", "with length:", len(current_seq))
        print("indexes:",current_idxs, "with length:", len(current_idxs))
        print("suceptability",current_SRs, "with length:", len(current_SRs))
        print('error at', j)
        print("--------------------")
    current_idxs = [int(item[0]) for item in current_idxs]
    #for i in range(0,max_length[1] - len(current_idxs)):
    #    current_idxs.append(-1)
    #for i in range(0,max_length[1] - len(current_SRs)):
    #    current_SRs.append(-1)
    list_idx.append(current_idxs)
    list_SR.append(current_SRs)
for i in range(len(list_idx)):
    if len(list_idx[i]) != len(list_SR[i]):
        print('error at', i)

In [9]:
import torch
import torch.nn.functional as F

def custom_loss(logits, targets, pad_index=-1):
    loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
    
    mask = (targets != pad_index).float()
    masked_loss = loss * mask
    
    average_loss = masked_loss.sum() / mask.sum()
    
    return average_loss


In [24]:
from torch.utils.data import DataLoader 
from bert_builder import BERT_ft
import torch.nn as nn
import copy
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_function = torch.nn.BCEWithLogitsLoss()
bert_test = BERT_ft(len(vocabulary_geno), max_length, dim_emb, dim_hidden, attention_heads, num_encoders, drop_prob, len(vocabulary_pheno), device)

optimizer = torch.optim.AdamW(bert_test.parameters(), lr=0.001, weight_decay=0.01)
criterion = nn.CrossEntropyLoss(ignore_index = -1).to(device)


loader = DataLoader(test_set, batch_size=16, shuffle=False)
for i, batch in enumerate(loader):
    if i >= 10:
        break 
    optimizer.zero_grad()
    input, token_target, attn_mask, AB_idx, SR_class = batch
    token_predictions, resistance_predictions = bert_test(input, attn_mask) 
    result_list = []
    for j in range(len(AB_idx)):
        result_tensor = torch.full((81,), -1)  # Create tensor filled with -1 values
        for idx, value in enumerate(AB_idx[j]):
            if value != -1:
                result_tensor[value.item()] = SR_class[j][idx]
                #print(AB_idx[j])
                #print(SR_class[j])
                #print(result_tensor)
        result_list.append(result_tensor)
    ab_loss = 0
    pheno_loss = 0
    for i, row in enumerate(resistance_predictions):
        prediction = row
        #print("pred",prediction, "shape ", prediction.shape,"type" ,type(prediction))
        target = result_list[i]
        #print("target",target, "shape ", target.shape, "type", type(target))
        ab_loss = custom_loss(prediction, target.float()) 
        print(ab_loss)
        pheno_loss += ab_loss
    print("pheno",pheno_loss)
    params_before_optimization = copy.deepcopy(bert_test.state_dict())
    pheno_loss.backward()
    # Call optimizer.step() to perform optimization
    optimizer.step()

    # Compare parameter values after optimization
    params_after_optimization = bert_test.state_dict()

    # Check if parameters were updated
    parameters_updated = any((params_before_optimization[key] != params_after_optimization[key]).any() for key in params_before_optimization)

    if parameters_updated:
        print("Parameters were updated after optimization.")
    else:
        print("Parameters were not updated after optimization.")
    

target = torch.tensor([ 0, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1,
        -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
        -1, -1,  0, -1, -1, -1,  0,  0, -1, -1,  1, -1, -1,  0, -1, -1, -1,  1,
        -1, -1, -1,  0, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         1, -1, -1,  0,  0, -1,  1, -1, -1])
prediction = torch.tensor([ 0.4786,  0.4658,  0.2647,  1.0500,  0.5575,  0.1078, -0.0405, -0.3351,
        -0.6943, -0.5706, -1.3000, -0.0464,  0.1630,  0.0561,  1.1942, -0.7140,
        -0.4864,  0.1857,  0.5022,  0.3904, -1.0160,  0.5669, -0.2253, -0.2588,
        -0.5341, -0.0920, -0.8664,  0.3386,  0.3392, -0.5458, -0.3611, -0.9976,
         0.6472, -0.2721, -1.0072, -0.2907,  0.6698, -0.0895,  0.9034,  0.2583,
        -1.0019, -0.6530,  0.2442,  0.3329, -0.5051, -0.1699,  0.4667,  0.2443,
        -0.2963, -0.2937, -0.6594, -0.5588,  0.5937,  1.1591, -0.1682, -0.3226,
         0.8355,  0.2033,  0.1068, -0.2998,  0.3313, -0.1574,  0.1974, -1.0022,
         0.7160,  0.5348, -0.6237,  0.7474, -0.1159, -0.4257, -0.6888, -0.4665,
        -0.5394, -0.5916, -0.3950,  0.0524,  0.4405, -0.3407, -0.5185, -0.5657,
        -0.3095])
print("------------------")
print("target",target, "shape ", target.shape, "type", type(target))
print("pred",prediction, "shape ", prediction.shape,"type" ,type(prediction))
loss = loss_function(prediction, target.float())
print(loss)

tensor(0.4494, grad_fn=<DivBackward0>)
tensor(0.6192, grad_fn=<DivBackward0>)
tensor(0.4631, grad_fn=<DivBackward0>)
tensor(0.8214, grad_fn=<DivBackward0>)
tensor(0.8458, grad_fn=<DivBackward0>)
tensor(0.6789, grad_fn=<DivBackward0>)
tensor(0.7272, grad_fn=<DivBackward0>)
tensor(0.6520, grad_fn=<DivBackward0>)
tensor(0.7455, grad_fn=<DivBackward0>)
tensor(0.6448, grad_fn=<DivBackward0>)
tensor(0.7252, grad_fn=<DivBackward0>)
tensor(0.7435, grad_fn=<DivBackward0>)
tensor(0.6430, grad_fn=<DivBackward0>)
tensor(0.6515, grad_fn=<DivBackward0>)
tensor(0.6933, grad_fn=<DivBackward0>)
tensor(0.6552, grad_fn=<DivBackward0>)
pheno tensor(10.7591, grad_fn=<AddBackward0>)
Parameters were updated after optimization.
tensor(0.6246, grad_fn=<DivBackward0>)
tensor(0.5742, grad_fn=<DivBackward0>)
tensor(0.7922, grad_fn=<DivBackward0>)
tensor(0.5629, grad_fn=<DivBackward0>)
tensor(0.6612, grad_fn=<DivBackward0>)
tensor(0.6294, grad_fn=<DivBackward0>)
tensor(0.6250, grad_fn=<DivBackward0>)
tensor(0.5781

In [ ]:
list_AB_predictions = []
    for i, row in enumerate(resistance_predictions):
        AB_list = 0
        AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
        current_abs = []
        for ab in AB_list:
            current_abs.append(row[ab].item())
        current_abs = torch.tensor(current_abs, requires_grad=True)
        list_AB_predictions.append(current_abs)
    
    processed_tensor = [row[row != -1] for row in SR_class]
    ab_loss = 0
    for i, row in enumerate(processed_tensor):
        row = torch.tensor(row, dtype=torch.float32,requires_grad=True)
        list_AB_predictions[i] = torch.tensor(list_AB_predictions[i], dtype=torch.float32,requires_grad=True)

        ab_loss += self.ab_criterion(list_AB_predictions[i], row)

In [ ]:
test = res_pred[14]
print(test)
pred_res = torch.where(test > 0, torch.ones_like(test), torch.zeros_like(test))
print(pred_res)
indicies = [1,2,3,4]
test = pred_res[indicies]
print)

Trainer

In [ ]:
import numpy as np
from create_dataset import NCBIDataset
def get_split_indices(size_to_split, val_share, random_state: int = 42):
    indices = np.arange(size_to_split)
    np.random.seed(random_state)
    np.random.shuffle(indices)
    
    train_share = 1 - val_share
    
    train_size = int(train_share * size_to_split)
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    
    return train_indices, val_indices
max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob)

In [13]:
import time
from torch import nn
from torch.utils.data import DataLoader
import wandb
from pathlib import Path
import copy
import torch.nn.functional as F

class BertTrainer_ft:
    def __init__(self, model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, wandb_mode, project_name, wandb_name):
        
        self.model = model
        self.train_set = train_set
        self.train_size = len(train_set)
        self.val_set = val_set
        self.epochs = epochs    
        self.batch_size = batch_size
        self.num_batches = self.train_size // self.batch_size
        self.lr = lr
        self.weight_decay = 0.1
        self.current_epoch  = 0
        self.early_stopping_counter = 0	
        self.patience = stop_patience
        
        self.wandb_mode = wandb_mode
        self.project_name = project_name
        self.wandb_name = wandb_name
        self.device = device
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.token_criterion = nn.CrossEntropyLoss(ignore_index = -1).to(self.device)
        self.ab_criterion = nn.BCEWithLogitsLoss().to(self.device)


    def __call__(self):   
        if self.wandb_mode:
            self._init_wandb()   
        self.val_set.prepare_dataset() 
        self.val_loader = DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
        start_time = time.time()
        self.best_val_loss = float('inf')
        self._init_result_lists()
        for self.current_epoch in range(self.current_epoch, self.epochs):
            #Training
            self.model.train()
            self.train_set.prepare_dataset()
            self.train_loader = DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
            epoch_start_time = time.time()
            avg_epoch_loss_geno, avg_epoch_loss_pheno = self.train(self.current_epoch)
            self.train_losses_geno.append(avg_epoch_loss_geno) 
            self.train_losses_ab.append(avg_epoch_loss_pheno)  
            print(f"Epoch completed in {(time.time() - epoch_start_time)/60:.1f} min")
            
            #Validation
            print("Evaluating on validation set...")
            val_results = self.evaluate(self.val_loader)
            print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))}")
            self.val_losses_geno.append(val_results[0])
            self.val_losses_ab.append(val_results[1])
            self.val_accs.append(val_results[2])
            if self.wandb_mode:
                self._report_epoch_results()
            criterion = self.stop_early()
            if criterion:
                print(f"Training interrupted at epoch: {self.current_epoch+1}")
                break
        print(f"-=Training completed=-")
        results = {
            "best_epoch": self.best_epoch,
            "geno_train_losses": self.train_losses_geno,
            "ab_train_losses": self.train_losses_ab,
            "geno_val_losses": self.val_losses_geno,
            "ab_val_losses": self.val_losses_ab,
            "val_accs": self.val_accs
        }
        return results

    def _init_result_lists(self):

        self.train_losses_geno = []
        self.train_losses_ab = []

        self.val_losses_geno = []
        self.val_losses_ab = []

        self.val_accs = []
    
    def stop_early(self):
        if self.val_losses_ab[-1] < self.best_val_loss:
            self.best_val_loss = self.val_losses_ab[-1]
            self.best_epoch = self.current_epoch
            self.best_model_state = self.model.state_dict()
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return True if self.early_stopping_counter >= self.patience else False

    def train(self, epoch: int):
        print(f"Epoch {epoch+1}/{self.epochs}")
        time_ref = time.time()
        
        epoch_loss_geno = 0
        epoch_loss_pheno = 0

        for i, batch in enumerate(self.train_loader):
            input, token_target, attn_mask, AB_idx, SR_class = batch
            
            ABinclusion = torch.unique(AB_idx)
            ABinclusion = ABinclusion[ABinclusion != -1]
            ABinclusion = ABinclusion.tolist()
            self.model.exclude_networks(ABinclusion)

            self.optimizer.zero_grad() 

            token_predictions, resistance_predictions = self.model(input, attn_mask) 
            geno_loss = self.token_criterion(token_predictions.transpose(-1, -2), token_target) 
            
            result_list = []
            for j in range(len(AB_idx)):
                result_tensor = torch.full((81,), -1) 
                for idx, value in enumerate(AB_idx[j]):
                    if value != -1:
                        result_tensor[value.item()] = SR_class[j][idx]
                result_list.append(result_tensor)
            ab_loss = 0
            pheno_loss = 0
            for i, row in enumerate(resistance_predictions):
                prediction = row
                target = result_list[i]
                ab_loss = custom_loss(prediction, target.float()) 
                pheno_loss += ab_loss
            pheno_loss.backward() 
            epoch_loss_geno += geno_loss.item()
            epoch_loss_pheno += pheno_loss.item()

            self.optimizer.step()
            self.model.reset_exclusion()   
              

        avg_epoch_loss_geno = epoch_loss_geno / self.num_batches
        avg_epoch_loss_pheno = epoch_loss_pheno / self.num_batches

        return avg_epoch_loss_geno, avg_epoch_loss_pheno

    def custom_loss(logits, targets, pad_index=-1):
        loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
        
        mask = (targets != pad_index).float()
        masked_loss = loss * mask
        
        average_loss = masked_loss.sum() / mask.sum()
        
        return average_loss

    
    def evaluate(self, loader):
        self.model.eval()
        epoch_loss_geno = 0
        epoch_loss_ab = 0
        total_correct = 0
        total_sum = 0
  
        with torch.no_grad():
            for i, batch in enumerate(loader):
                input, token_target, attn_mask, AB_idx, SR_class = batch

                token_predictions, resistance_predictions = self.model(input, attn_mask) 
                geno_loss = self.token_criterion(token_predictions.transpose(-1, -2), token_target) 
                
                result_list = []
                for j in range(len(AB_idx)):
                    result_tensor = torch.full((81,), -1)  # Create tensor filled with -1 values
                    for idx, value in enumerate(AB_idx[j]):
                        if value != -1:
                            result_tensor[value.item()] = SR_class[j][idx]
                    result_list.append(result_tensor)
                ab_loss = 0
                pheno_loss = 0
                for i, row in enumerate(resistance_predictions):
                    prediction = row
                    target = result_list[i]
                    ab_loss = custom_loss(prediction, target.float()) 
                    pheno_loss += ab_loss
                epoch_loss_geno += geno_loss.item()
                epoch_loss_ab += pheno_loss.item() 
                
                list_AB_predictions = []
                for i, row in enumerate(resistance_predictions):
                    AB_list = 0
                    AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
                    current_abs = []
                    for ab in AB_list:
                        current_abs.append(row[ab].item())
                    current_abs = torch.tensor(current_abs)
                    current_abs = current_abs.type(torch.int16)
                    list_AB_predictions.append(current_abs)
                
                    processed_tensor = [row[row != -1] for row in SR_class]
                for i, row in enumerate(processed_tensor):
                    total_correct += (row == list_AB_predictions[i]).sum().item()
                    total_sum += len(row)

        avg_epoch_loss_geno = epoch_loss_geno / self.num_batches
        avg_epoch_loss_ab = epoch_loss_ab / self.num_batches

        accuracy = total_correct / total_sum

        return avg_epoch_loss_geno, avg_epoch_loss_ab, accuracy
    
    def _save_model(self, savepath: Path):
        torch.save(self.best_model_state, savepath)
        print(f"Model saved to {savepath}")
        
        
    def _load_model(self, savepath: Path):
        print(f"Loading model from {savepath}")
        self.model.load_state_dict(torch.load(savepath))
        print("Model loaded")

    def _init_wandb(self):
        self.wandb_run = wandb.init(
            project=self.project_name, # name of the project
            name=self.wandb_name, # name of the run
            
            config={
                "epochs": self.epochs,
                "batch_size": self.batch_size,
                "num_heads": self.model.attention_heads,
                "num_encoders": self.model.num_encoders,
                "emb_dim": self.model.dim_embedding,
                'ff_dim': self.model.dim_embedding,
                "lr": self.lr,
                "weight_decay": self.weight_decay,
                "max_seq_len": self.model.max_length[0],
                "vocab_size": len(self.train_set.vocab_geno),
                "num_parameters": sum(p.numel() for p in self.model.parameters() if p.requires_grad),
            }
        )
        self.wandb_run.watch(self.model) # watch the model for gradients and parameters
        self.wandb_run.define_metric("epoch", hidden=True)
        self.wandb_run.define_metric("batch", hidden=True)

        self.wandb_run.define_metric("GenoLosses/geno_train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("GenoLosses/geno_val_loss", summary="min", step_metric="epoch")

        self.wandb_run.define_metric("AB_Losses/ab_train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("AB_Losses/ab_val_loss", summary="min", step_metric="epoch")

        self.wandb_run.define_metric("Accuracies/val_acc", summary="min", step_metric="epoch")
        
        self.wandb_run.define_metric("Losses/final_val_loss")
        self.wandb_run.define_metric("Accuracies/final_val_acc")
        self.wandb_run.define_metric("final_epoch")

        return self.wandb_run
    
    def _report_epoch_results(self):
        wandb_dict = {
            "epoch": self.current_epoch+1,
            
            "GenoLosses/geno_train_loss": self.train_losses_geno[-1],
            "ABLosses/ab_train_loss": self.train_losses_ab[-1],

            "GenoLosses/geno_val_loss": self.val_losses_geno[-1],
            "ABLosses/ab_val_loss": self.val_losses_ab[-1],
            
            "Accuracies/val_acc": self.val_accs[-1],
        }
        self.wandb_run.log(wandb_dict)

In [ ]:
from data_preprocessing import data_loader
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from misc import get_split_indices
from create_dataset import NCBIDataset
include_pheno = False

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)


include_pheno = False
max_length = [51,81]
mask_prob = 0.25
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

test_set = NCBIDataset(NCBI, vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
test_set.prepare_dataset()

In [ ]:
from bert_builder import BERT_pt
from trainer import BertTrainer_pt

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERT_pt(vocab_size=len(vocabulary_geno), max_length=51, dim_embedding = 128, attention_heads=8, num_encoders=2, dropout_prob=0.2)
save_directory = 'c:\\Users\\erika\\Desktop\\Exjobb\\savefiles'

trainer = BertTrainer_pt(model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, save_directory)
trainer()

In [5]:
from bert_builder import BERT_ft
from data_preprocessing import data_loader
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from misc import get_split_indices
from create_dataset import NCBIDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lr = 0.001

include_pheno = True

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

#reduced_samples = 4000
#NCBI = NCBI.head(reduced_samples)

print(f"Data loaded, {len(NCBI)} samples found")
print(f"length  of token vocabulary:",len(vocabulary_geno))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)


Data loaded, 6483 samples found
length  of token vocabulary: 475


In [14]:
lr = 0.0001
bert_test = BERT_ft(len(vocabulary_geno), max_length, dim_emb, dim_hidden, attention_heads, num_encoders, drop_prob, len(vocabulary_pheno), device)

test = BertTrainer_ft(bert_test, train_set, val_set, 30, batch_size, lr, device, stop_patience,  True, "Trial_runs", "MaxSamples_2")
test()

ABLosses/ab_train_loss,█▄▃▃▂▂▂▁▁▁▁▁
ABLosses/ab_val_loss,█▆▅▄▃▂▂▁▁▃▁▂
Accuracies/val_acc,█▅▆▃▃▁▃▄▅▄▄▄
GenoLosses/geno_train_loss,▇█▃▆▄▄▃▅▁▁▁
GenoLosses/geno_val_loss,█▄█▇▆▄▄▅▃▁▄▅
epoch,▁▂▂▃▄▄▅▅▆▇▇█
ABLosses/ab_train_loss,8.47424
ABLosses/ab_val_loss,2.06436
epoch,12


Epoch 1/30
Epoch completed in 0.6 min
Evaluating on validation set...
Elapsed time: 00:00:42
Epoch 2/30
Epoch completed in 0.6 min
Evaluating on validation set...
Elapsed time: 00:01:24
Epoch 3/30
Epoch completed in 0.6 min
Evaluating on validation set...
Elapsed time: 00:02:06
Epoch 4/30
Epoch completed in 0.6 min
Evaluating on validation set...
Elapsed time: 00:02:49
Epoch 5/30
Epoch completed in 0.6 min
Evaluating on validation set...
Elapsed time: 00:03:32
Epoch 6/30
Epoch completed in 0.6 min
Evaluating on validation set...
Elapsed time: 00:04:15
Epoch 7/30
Epoch completed in 0.6 min
Evaluating on validation set...
Elapsed time: 00:04:58
Epoch 8/30
Epoch completed in 0.6 min
Evaluating on validation set...
Elapsed time: 00:05:41
Epoch 9/30
Epoch completed in 0.6 min
Evaluating on validation set...
Elapsed time: 00:06:24
Epoch 10/30
Epoch completed in 0.7 min
Evaluating on validation set...
Elapsed time: 00:07:08
Epoch 11/30
Epoch completed in 0.6 min
Evaluating on validation set..

{'best_epoch': 22,
 'geno_train_losses': [6.509485895251051,
  nan,
  6.45091062710609,
  6.452215127003046,
  6.439010040259656,
  6.451745283456496,
  6.431238848486064,
  6.447394635942247,
  6.43535570450771,
  6.438558978798949,
  nan,
  6.42722154841011,
  6.433914914543246,
  6.411501357584823,
  6.431412431928846,
  nan,
  6.402492025752126,
  nan,
  6.398771545033396,
  6.394085301293267,
  6.404544156274678,
  nan,
  6.38123443391588,
  6.395017715147984,
  6.38099784909943,
  nan,
  6.391568416430626,
  6.3692129482457664],
 'ab_train_losses': [13.942978940260263,
  11.35505305028256,
  10.671379770393726,
  10.255622599043964,
  9.982498461265623,
  9.783324732456679,
  9.648679252024051,
  9.51229977791692,
  9.389829956822926,
  9.278195781104358,
  9.24832558558311,
  9.194965418474174,
  9.059573783182804,
  9.055769923108596,
  9.068263903076266,
  8.987652490536371,
  8.936795033422516,
  8.893707693358998,
  8.80360415338734,
  8.802425288859709,
  8.811089030754419,

------------------------------

In [58]:
import torch

# Example tensors
AB_idx = torch.tensor([49, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                       -1, -1, -1, -1, -1, -1, -1, -1, -1])

SR_class = torch.tensor([1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                         -1, -1, -1, -1, -1, -1, -1, -1, -1])

# Create a new tensor filled with -1 values
result_tensor = torch.full_like(AB_idx, -1)

# Iterate over AB_idx and SR_class to place the values at correct indices
for idx, value in enumerate(AB_idx):
    if value != -1:
        result_tensor[value] = SR_class[idx]

print(result_tensor)

tensor([-1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1])
